<a id="top"></a>
# Resolving Named Objects with Astroquery
***
## Learning Goals

- Understand how `astroquery.mast` resolves named astronomical objects.
- Use the `resolve_object` function to resolve named objects.
- Compare the results of different resolvers.
- Configure the resolver used in `astroquery.mast` queries.

## Table of Contents
* [Introduction](#Introduction)
* [Resolving an Object](#Resolving-an-Object)
    * [Specifying a Resolver](#Specifying-a-Resolver)
* [Comparing Results Across Resolvers](#Comparing-Results-Across-Resolvers)
* [Configuring the Resolver in Queries](#Configuring-the-Resolver-in-Queries)
* [Additional Resources](#Additional-Resources)

## Introduction

When querying the [Mikulski Archive for Space Telescopes (MAST)](https://archive.stsci.edu/) using [`astroquery.mast`](https://astroquery.readthedocs.io/en/latest/mast/mast.html), it's common to search for data by specifying the name of a well-known astronomical object—such as [“M31”](https://simbad.u-strasbg.fr/simbad/sim-id?Ident=M31) or [“Betelgeuse”](https://simbad.u-strasbg.fr/simbad/sim-id?Ident=Betelgeuse)—rather than providing precise celestial coordinates. For an in-depth discussion of how name resolution works, including potential pitfalls, resolver order, and real-world examples, see the ["Searching for Named Sources"](https://outerspace.stsci.edu/display/MASTDOCS/Searching+for+Named+Sources) article in the MAST documentation.

These name-based queries are handled by [SANTA, the STScI Archive Name Translation Application](https://mastresolver.stsci.edu/Santa-war/). SANTA resolves object names into celestial coordinates by querying a sequence of resolver services—including [SIMBAD](https://simbad.u-strasbg.fr/simbad/), [NED](https://ned.ipac.caltech.edu/), [TIC](https://tess.mit.edu/science/tess-input-catalogue/), and others—until a match is found or all options are exhausted. Different resolvers may return slightly different positions for the same object due to variations in reference catalogs. These differences can impact which observations are returned from MAST, particularly when searching for angularly large sources using a narrow search radius. More dramatically, different resolvers may return completely different objects for the same name due to differing naming conventions. See the 
["Problem cases" section of "Searching for Named Sources"](https://outerspace.stsci.edu/display/MASTDOCS/Searching+for+Named+Sources#SearchingforNamedSources-Problemcases) for more information.

In this notebook, we will explore how to use `astroquery.mast` to resolve named objects and configure the resolver used in our queries.

## Imports

This notebook uses the following packages:
- *astroquery.mast* to resolve named astronomical objects and query data.

In [1]:
from astroquery import log
from astroquery.mast import Observations

***

## Resolving an Object

First, we'll demonstrate how to interact directly with the SANTA service using `astroquery.mast` before showing how to incorporate these settings into MAST queries.

When working with data from MAST, it's often helpful to resolve the name of an astronomical object into its celestial coordinates. The [`resolve_object`](https://astroquery.readthedocs.io/en/latest/mast/mast.html#resolving-object-names) function, available in any of the MAST query classes, provides a flexible way to do this. It queries the SANTA service and supports specifying a particular resolver or retrieving results from all available resolvers.

Let's resolve an object name using SANTA's default resolver sequence. We'll use the `Observations` class to do this, but you could also use `Catalogs`, `MastMissions`, or any other MAST query class. Below, we will resolve the name ["NGC 1909"](https://theskylive.com/sky/deepsky/ngc1909-witch-head-nebula-object), which refers to a [reflection nebula](https://www.nasa.gov/image-article/reflection-nebula/) in the constellation of [Eridanus](https://noirlab.edu/public/education/constellations/eridanus/).

In [2]:
log.setLevel("DEBUG")
Observations.resolve_object("NGC 1909")
log.setLevel("INFO")

DEBUG: Coordinates resolved using SIMBAD: <SkyCoord (ICRS): (ra, dec) in deg
    (75.5, -7.9)> [astroquery.mast.utils]


As you can tell from the debug statement, this object was resolved using the SIMBAD resolver. This means that SIMBAD was the first resolver in the sequence to return a result.

### Specifying a Resolver

If you want to use a specific resolver, you can do so by passing the `resolver` argument to the `resolve_object` function. The available options for `resolver` are `SIMBAD` and `NED` (case-insensitive). At this time, other resolvers such as TIC cannot be specified here. Below, we will resolve "NGC 1909" using both the SIMBAD and NED resolvers. The results will be slightly different, as each resolver uses a different reference catalog and naming convention.

In [3]:
simbad_coord = Observations.resolve_object("NGC 1909", resolver="simbad")
ned_coord = Observations.resolve_object("NGC 1909", resolver="ned")

print("SIMBAD Coordinates: ", simbad_coord)
print("NED Coordinates: ", ned_coord)

# Print the separation between the two coordinates
print(f"Separation: {simbad_coord.separation(ned_coord).arcmin} arcmin")

SIMBAD Coordinates:  <SkyCoord (ICRS): (ra, dec) in deg
    (75.5, -7.9)>
NED Coordinates:  <SkyCoord (ICRS): (ra, dec) in deg
    (76.23101, -7.26563)>
Separation: 57.78376792763028 arcmin


The separation between the two resolved positions is over 55 arcminutes, which is significant enough to affect the results of a query. For example, if you were to search for observations within a 10 arcminute radius of the resolved position, you would get different results depending on which resolver you used.

## Comparing Results Across Resolvers

To compare the results across all available resolvers in SANTA, we can set the `resolve_all` parameter to `True`. This will return dictionary where the keys are the names of the resolvers and the values are the resolved positions. This is useful for understanding how different resolvers interpret the same object name and can help you choose the best resolver for your needs.

Let's query all resolvers for the object "mu eri". This is a known [imprecise source name](https://outerspace.stsci.edu/display/MASTDOCS/Searching+for+Named+Sources#SearchingforNamedSources-Problemcases), meaning that the identifier refers to a different astrophysical source based on the resolver used. You'll notice that the coordinates returned by the SIMBAD and NED resolvers are quite different.

In [4]:
coords = Observations.resolve_object('mu eri', resolve_all=True)
coords

{'SIMBAD': <SkyCoord (ICRS): (ra, dec) in deg
     (71.37563095, -3.25465981)>,
 'NED': <SkyCoord (ICRS): (ra, dec) in deg
     (42.04403, -15.30113)>}

In [5]:
# Print the separation between the coordinates returned by SIMBAD and NED
print(f"Separation: {coords['SIMBAD'].separation(coords['NED']).arcmin} arcmin")

Separation: 1877.5835379664597 arcmin


For some variety, let's also resolve an object from the TESS Input Catalog (TIC).

In [6]:
Observations.resolve_object("TIC 141914082", resolve_all=True)

{'TIC': <SkyCoord (ICRS): (ra, dec) in deg
     (94.6175354, -72.04484622)>,
 'SIMBAD': <SkyCoord (ICRS): (ra, dec) in deg
     (94.61753263, -72.04484665)>}

## Configuring the Resolver in Queries

Because the resolver used in a query can affect the results returned, it's important to be able to configure the resolver used in your queries. Query methods that accept a `resolver` argument will use the specified resolver when resolving object names. If you don't specify a resolver, the default resolver sequence will be used.

Let's see this in action. We'll query the MAST Archive for observations of "NGC 1909" using the Simbad resolver, and then we'll do the same using the NED resolver. The results should be different, as we expect the two resolvers to return different positions for the same object name. We'll set our search radius to 2 arcminutes.

In [7]:
# Query NGC 5658 with SIMBAD
simbad_obs = Observations.query_object("NGC 1909", radius="2 arcmin", resolver="simbad")
print("Number of results from SIMBAD: ", len(simbad_obs))

# Query NGC 5658 with NED
ned_obs = Observations.query_object("NGC 1909", radius="2 arcmin", resolver="ned")
print("Number of results from NED: ", len(ned_obs))

# Many results correspond to individual dithers within a single Spitzer observation.
# We can count the number of unique observations by creating a set of the observation IDs.
obs_ids_simbad = set(simbad_obs['obs_id'])
obs_ids_ned = set(ned_obs['obs_id'])
print("Number of unique observations from SIMBAD: ", len(obs_ids_simbad))
print("Number of unique observations from NED: ", len(obs_ids_ned))

# Get the difference between the two sets of observation IDs
disjoint_obs_ids = obs_ids_simbad - obs_ids_ned
print(f"Number of unique observations in SIMBAD not in NED: {len(disjoint_obs_ids)}")

Number of results from SIMBAD:  33


Number of results from NED:  35
Number of unique observations from SIMBAD:  33
Number of unique observations from NED:  35
Number of unique observations in SIMBAD not in NED: 28


The coordinates resolved by NED return many more observations than those resolved by SIMBAD. There are also some observations that are returned by SIMBAD and not by NED, and vice versa. When searching MAST, specifying a resolver can help to make queries more consistent and reproducible. By explicitly choosing a resolver, you can avoid any surprises that might arise from changes in the default resolver sequence or differences in the results returned by different resolvers.

The following MAST query methods accept a `resolver` argument:
- `Observations.query_object`
- `Observations.query_object_count`
- `Observations.query_criteria`
- `Observations.query_criteria_count`
- `Catalogs.query_object`
- `Catalogs.query_criteria`
- `MastMissions.query_object`
- `MastMissions.query_criteria`
- `Tesscut.get_sectors`
- `Tesscut.download_cutouts`
- `Tesscut.get_cutouts`

## Additional Resources

- [Astroquery.mast Documentation](https://astroquery.readthedocs.io/en/latest/mast/mast.html)
- [SANTA Documentation](https://mastresolver.stsci.edu/Santa-war/)

## Citations

If you use `astroquery` for published research, please cite the
authors. Follow these links for more information about citing `astroquery`:

* [Citing Astroquery](https://github.com/astropy/astroquery/blob/main/astroquery/CITATION)

## About this Notebook

**Author(s):** Sam Bianco (sbianco@stsci.edu) <br>
**Keyword(s):** Astroquery, resolver, SANTA <br>
**First published:** May 2025 <br>
**Last updated:** May 2025 <br>

***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/style-guides/master/guides/images/stsci-logo.png" alt="Space Telescope Logo" width="200px"/> 